A typical use of the ZTF alert archive will involve retrieving all alerts fulfilling certain criteria, e.g. occuring within a particular time, being sufficiently bright and so forth. These alert samples can be used both for archive based scientific inquires as well as for larger scale testing of algorithms prior to using them live.

This notebook demonstrates a cone-based search around specific coordinates, to which candidate quality metrics are added. This notebook uses stream query setup. This query does not immediately return a set of alerts but a _resume token_. This resume token can then be used to generate an iterator which obtaines chunked set of alerts. This setup allows for large alert samples to be processed without waiting for the archive processing or transferring a single large query result.


In [ ]:
import requests, os

In [ ]:
# This is the archive token which can be obtained from https://ampel.zeuthen.desy.de/live/dashboard/tokens
# In order to retrieve ZTF partnership alerts your token needs to have the appropriate access
token = os.environ["ARCHIVE_TOKEN"]   # I have mine stored
header = {"Authorization": "bearer "+token}

##### 1. Creating resume token based on alert properties. 

This notebooks demonstrates how to submit a query based on alert properties. The sample case includes cuts on time, ra, dec, RealBocus and number of detections. For more options, see the `ztf.alert.candidate` section of https://zwickytransientfacility.github.io/ztf-avro-alert/schema.html

In [ ]:
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'

In [ ]:
query = {
"cone": {
    "ra": 193.28143551564065,
    "dec": 33.488720966906016,
    "radius": 1. / 120
  },
"candidate": {
    "drb": {
      "$gt": 0.995
    },
    "ndethist": {
      "$gt": 1,
      "$lte": 10000
    },
    "isdiffpos": {"$in": ["t", "1"]},
  }
}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
if not response.ok:
    print( 'Query creation failed.')

In [ ]:
# The full response contains the resume token as well as the chunk size, i.e.
# how many alerts will be return in each call to the alert iterator.
response.json()

In [ ]:
resume_token = response.json()['resume_token']

At this point the alert archive will start the process of staging alerts for release. This process takes a few min (length depending on query size), during which time the resume_token will stay locked. 

##### 2. Retrieve alerts based on resume token

The next step will involve using a resume token to locally retrieve all alerts covered by the query. For this purpose we will use the `ZTFArchiveAlertLoader`, which provides an iterator with which we can traverse the alerts. As soon as one alert bunch is exhaused the next will be obtained from the archive. 



In [ ]:
from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader

In [ ]:
# The loader config contains the resume_token as stream identifier
config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", 
          "stream":resume_token}

In [ ]:
config

In [ ]:
try:
    alertlist = []
    alertloader = ZTFArchiveAlertLoader(**config)
    alerts = alertloader.get_alerts()
    for alert in alerts:
        alertlist.append(alert)
        print(alert['objectId'], alert['candidate']['jd'])
except requests.exceptions.HTTPError as e:
    status_code = e.response.status_code
    if status_code==423:
        print('HTTP error {}: likely caused by server staging process. Wait and try again.'.format(status_code) )
    else:
        raise e
      

##### We now have a list of alerts which can be examined further, e.g. by an AMPEL filter or processing unit.

In [ ]:
print('Found {} alerts'.format(len(alertlist)))